In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import numpy as np
import pandas as pd

import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import glob

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.get_logger().setLevel("ERROR")

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


In [ ]:
device_name = tf.test.gpu_device_name()
print(tf.test.gpu_device_name())


In [ ]:
import zipfile
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip') as zf:
    zf.extractall()
with zipfile.ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip') as zf:
    zf.extractall()


In [ ]:
import cv2

training_data_X = []
training_data_Y = []
IMG_SIZE = 224
DIR_PATH = '/kaggle/working/train'
for img in os.listdir(DIR_PATH):
    if 'dog.' == img[:4]:
        category = 1
    else:
        category = 0
        
    training_data_X.append(DIR_PATH + "/" + img)
    training_data_Y.append(category)

print(len(training_data_X))


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(training_data_X, training_data_Y, test_size = 0.3, random_state=50)


In [ ]:
def image_load(path, label):
    image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, tf.one_hot(label, 2)


In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))

ds_train = ds_train.map(image_load)
ds_val = ds_val.map(image_load)

print('train dataset:',len(ds_train),'validation dataset:',len(ds_val))


In [ ]:
batch_size = 64

ds_batch_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_batch_train = ds_batch_train.prefetch(tf.data.AUTOTUNE)
ds_batch_val = ds_val.batch(batch_size=batch_size, drop_remainder=True)


In [ ]:
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)


In [ ]:
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model


In [ ]:
model = build_model(num_classes=2)

epochs = 10  
history = model.fit(ds_batch_train, epochs=epochs, validation_data=ds_batch_val, verbose=1)


In [ ]:
history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values)+1)

line1 = plt.plot(epochs, val_loss_values, label ='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')
plt.setp(line1, linewidth = 2.0, marker='+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker='4', markersize=10.0)
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
history_dict = history.history

plt.plot(history_dict["accuracy"])
plt.plot(history_dict["val_accuracy"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "validation"], loc="upper left")
plt.grid(True)
plt.show()


In [ ]:
testing_data = []
testing_id = []

DIR_PATH = '/kaggle/working/test'
for img in os.listdir(DIR_PATH):
    testing_data.append(DIR_PATH + '/' + img)
    testing_id.append(int(img.split('.')[0]))

def test_image_load(path, id):
    image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, id

print(len(testing_data))


In [ ]:
ds_test = tf.data.Dataset.from_tensor_slices((testing_data, testing_id))

ds_test = ds_test.map(test_image_load)
ds_batch_test = ds_test.batch(batch_size=100, drop_remainder=True)


In [ ]:
from tqdm import tqdm
submission = {'id':[],'label':[]}
dog_prediction = lambda x:x[1]

for batch in tqdm(ds_batch_test):
    results = model.predict(batch[0])
    id = batch[1].numpy()
    
    submission['id'].extend(id)
    submission['label'].extend(map(dog_prediction,results))


In [ ]:
import pandas as pd

submission_df=pd.DataFrame(submission)
submission_df.to_csv('my_submission.csv',index=False)
